# Wikishop  
## Анализ комментариев на токсичность



In [80]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot  as plt
%matplotlib inline


import re
import string

#import torch
#import transformers 

import optuna
import lightgbm as lgb

from sklearn.model_selection import GridSearchCV


import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')

from sklearn.model_selection  import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import f1_score, mean_squared_error

import spacy
nlp = spacy.load('en_core_web_sm', disable=['parser','ner'])


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/juliagrobman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
!python -m spacy download en_core_web_sm

/opt/anaconda3/lib/python3.8/site-packages/cryptography/hazmat/backends/openssl/x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(
     |████████████████████████████████| 12.8 MB 443 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [13]:
pip install nltk


Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install spacy

/opt/anaconda3/lib/python3.8/site-packages/cryptography/hazmat/backends/openssl/x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(
     |████████████████████████████████| 6.5 MB 423 kB/s 
     |████████████████████████████████| 2.9 MB 360 kB/s 
     |████████████████████████████████| 768 kB 760 kB/s 
     |████████████████████████████████| 181 kB 667 kB/s 
     |████████████████████████████████| 457 kB 717 kB/s 
     |████████████████████████████████| 42 kB 301 kB/s 
     |████████████████████████████████| 106 kB 821 kB/s 
     |████████████████████████████████| 5.9 MB 403 kB/s 
     |████████████████████████████████| 58 kB 1.1 MB/s 
ERROR: dbt-core 0.21.0 has requirement packaging~=20.9, but you'll have packaging 21.3 which is incompatible.
ERROR: dbt-core 0.21.0 has requirement typing-extensions<3.11,>=3.7.4, but you'll have typing-extensions 4.3.0 which is incompatible.
  Atte

In [21]:
path_global = '/datasets/toxic_comments.csv'
path_local = '/Users/juliagrobman/Downloads/toxic_comments (1).csv'

if os.path.exists(path=path_global):
    data = pd.read_csv(path_global)
elif os.path.exists(path=path_local):
    data = pd.read_csv(path_local)
else: 
    print('ERROR IN PATH')

In [22]:
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [65]:
data['text'] = data['text'].astype('U')

### Text preprocessing

In [70]:
def  preprocess_text(text, stopwords):
    # cleaning punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # text to lower
    text = text.lower()

    # removing stopwords
    text = ' '.join([word for word in text.split() if word not in stopwords])

    # removing whitespaces
    text = re.sub(r'\s', ' ', text).strip()

    return text


In [71]:
def lemmatize_text(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])

In [72]:
data['clean'] = data['text'].apply(lambda x: preprocess_text(x, stopwords=stop_words))

In [73]:
data['lemmatized'] = data['clean'].apply(lambda x: lemmatize_text(x) )

In [74]:
data.head(20)

,text,toxic,clean,lemmatized
0,Explanation\nWhy the edits made under my usern...,0,explanation edits made username hardcore metal...,explanation edit make username hardcore metall...
1,D'aww! He matches this background colour I'm s...,0,daww matches background colour im seemingly st...,daww match background colour I m seemingly stu...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man im really trying edit war guy constant...,hey man I m really try edit war guy constantly...
3,"""\nMore\nI can't make any real suggestions on ...",0,cant make real suggestions improvement wondere...,can not make real suggestion improvement wonde...
4,"You, sir, are my hero. Any chance you remember...",0,sir hero chance remember page thats,sir hero chance remember page that s
5,"""\n\nCongratulations from me as well, use the ...",0,congratulations well use tools well · talk,congratulation well use tool well · talk
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,cocksucker piss around work,cocksucker piss around work
7,Your vandalism to the Matt Shirvington article...,0,vandalism matt shirvington article reverted pl...,vandalism matt shirvington article revert plea...
8,Sorry if the word 'nonsense' was offensive to ...,0,sorry word nonsense offensive anyway im intend...,sorry word nonsense offensive anyway I m inten...
9,alignment on this subject and which are contra...,0,alignment subject contrary dulithgow,alignment subject contrary dulithgow


In [75]:
features = data['lemmatized']
target = data['toxic']

In [76]:
features_train_full, features_valid, target_train_full, target_valid = train_test_split(features, target, test_size=0.2, shuffle=True)
features_train, features_test, target_train, target_test = train_test_split(features_train_full, target_train_full, test_size=0.2, shuffle=True)

In [77]:
count_tf_idf = TfidfVectorizer()
features_train = count_tf_idf.fit_transform(features_train)

In [78]:
features_valid = count_tf_idf.transform(features_valid)
features_test = count_tf_idf.transform(features_test)

In [81]:
params = {
    'penalty' : ['l1','l2'],
    'C': np.logspace(-3,3,7),
    'solver':['lbfg', 'liblinear']
}

model = LogisticRegression(random_state=12345)
grid = GridSearchCV(model, param_grid=params, scoring='f1')
grid.fit(features_train, target_train)
predictions = grid.predict(features_valid)
score = f1_score(predictions, target_valid)
score

/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 433, in _check_solver
    raise ValueError("Logistic Regression supports only solvers in %s, got"
ValueError: Logistic Regression supports only solvers in ['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga'], got lbfg.

  warnings.warn("Estimator fit failed. The score on this train-test"


0.77166392092257

In [82]:
grid.best_params_

{'C': 10.0, 'penalty': 'l1', 'solver': 'liblinear'}

In [84]:
# Проверим на тестовой выборке:
predictions = grid.predict(features_test)


In [88]:
score_test = f1_score(target_test, predictions)
print('F1 score {:.2f}'.format(score_test))

F1 score 0.79


In [90]:
test_sent = 'look at yourself, you are a fat, black, sodding idiot, thinking just about money'

In [91]:
test_sent_0 = 'I will surely recommend this product to my family and friends, thanks!'

In [92]:
test_sent = preprocess_text(test_sent, stop_words)
test_sent = lemmatize_text(test_sent)

In [93]:
test_sent_0 = preprocess_text(test_sent_0, stop_words)
test_sent_0 = lemmatize_text(test_sent_0)

In [94]:
test_sent_0

'surely recommend product family friend thank'

In [95]:
test = count_tf_idf.transform([test_sent])

In [96]:
test_0 = count_tf_idf.transform([test_sent_0])

In [97]:
grid.predict(test)

array([1])

In [98]:
grid.predict(test_0)

array([0])

### Наша модель хорошо предсказывает токсичные комментарии, f1 выше трешхолда в 0.75

In [101]:
## Попробуем LightGBM
train_data  = lgb.Dataset(features_train, label=target_train)
test_data = lgb.Dataset(features_test, label=target_test)

parameters = {
    'application': 'binary',
    'objective': 'binary',
    'metric': 'f1',
    'boosting': 'dart',
    'num_leaves': 31,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'learning_rate': 0.05,
    'verbose': 0
}

classifier = lgb.train(
    parameters,
    train_data,
    valid_sets=test_data,
    num_boost_round=5000,
    early_stopping_rounds=100,
    
)
predict = classifier.predict(features_valid)


/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] objective is set=binary, application=binary will be ignored. Current value: objective=binary
[LightGBM] [Warning] objective is set=binary, application=binary will be ignored. Current value: objective=binary
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.610606 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] objective is set=binary, application=binary will be ignored. Current value: objective=binary


/opt/anaconda3/lib/python3.8/site-packages/lightgbm/callback.py:223: UserWarning: Early stopping is not available in dart mode
  _log_warning('Early stopping is not available in dart mode')


In [ ]:
score_light = f1_score(target_valid, predict)

## Попробуем BERT

In [192]:
tokenizer = transformers.BertTokenizer(
    vocab_file='/Users/ulia/Downloads/uncased_L-8_H-512_A-8/vocab.txt')

tokenized = data['lemmatized'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

In [195]:
config = transformers.BertConfig.from_json_file(
    '/Users/ulia/Downloads/uncased_L-8_H-512_A-8/bert_config.json'
)

In [196]:
model_bert = transformers.BertModel.from_pretrained('/Users/ulia/Downloads/uncased_L-8_H-512_A-8/bert_model.ckpt.index', config=config)

OSError: Unable to load weights from pytorch checkpoint file for '/Users/ulia/Downloads/uncased_L-8_H-512_A-8/bert_model.ckpt.index' at '/Users/ulia/Downloads/uncased_L-8_H-512_A-8/bert_model.ckpt.index'. If you tried to load a PyTorch model from a TF 2.0 checkpoint, please set from_tf=True.